In [1]:
import pandas as pd

from bokeh.io import show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper
)
import bokeh.palettes as palset
from bokeh.plotting import figure

from bokeh.sampledata.us_counties import data as counties

In [6]:
df = pd.read_csv('../county-data.csv', index_col=0)

In [8]:
palette = palset.RdBu10
#palette.reverse()

counties = {
    code: county for code, county in counties.items() if (code[0]*1000+code[1]) in df.index
}


county_xs = [county["lons"] for county in counties.values()]
county_ys = [county["lats"] for county in counties.values()]

color_mapper = LogColorMapper(palette=palette)

county_names = df.county + ', ' + df.state
claim_rates =  (df.partd_claims_opioid_2014 / df.partd_claims_all_2014 * 100).fillna("unknown")
pop = df['pop']
poverty_rates = (df.SAIPE_poverty/df['pop']*100).fillna("unknown")
disability_rates = (df.SSI_blind_disabled / df['pop']*100).fillna("unknown")
opioid_deaths = df.CM_deaths_opiates.fillna("unknown")

source = ColumnDataSource(data=dict(
    x=county_xs,
    y=county_ys,
    name=county_names,
    rate=claim_rates,
    population=pop,
    poverty=poverty_rates,
    disability=disability_rates,
    opioids=opioid_deaths
))

TOOLS = "pan,wheel_zoom,reset,hover,save"

In [9]:
p = figure(
    title="Medicare Part D Opioid Claim Rate (opioid claims as a percentage of all claims)", tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    plot_width=800, plot_height=500
)
p.grid.grid_line_color = None

p.patches('x', 'y', source=source,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@name"),
    ("Medicare Part D opioid claim rate", "@rate%"),
    ("Population", "@population"),
    ("Poverty rate", "@poverty%"),
    ("Disabilty rate", "@disability%"),
    ("Opioid deaths", "@opioids"),
]

show(p)

In [ ]:
p = figure(
    title="Disability Rate (SSI disability claims/population)", tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    plot_width=800, plot_height=500
)
p.grid.grid_line_color = None

p.patches('x', 'y', source=source,
          fill_color={'field': 'disability', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@name"),
    ("Medicare Part D opioid claim rate", "@rate%"),
    ("Population", "@population"),
    ("Poverty rate", "@poverty%"),
    ("Disabilty rate", "@disability%"),
    ("Opioid deaths", "@opioids"),
]

show(p)